In [1]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

In [8]:
tes, sr=librosa.load("tes.wav", sr=None)
ten, sr=librosa.load("ten.wav", sr=None)
tex = tes+ten

trs, sr=librosa.load("trs.wav", sr=None)
trn, sr=librosa.load("trn.wav", sr=None)
trx = trs+trn

In [11]:
sz = 1024
hp = 512
wn = 'hann'

In [46]:
trS = abs(librosa.stft(trs, n_fft=sz, hop_length=hp, window=wn))
trN = abs(librosa.stft(trn, n_fft=sz, hop_length=hp, window=wn))
trX = abs(librosa.stft(trx, n_fft=sz, hop_length=hp, window=wn))
teX = librosa.stft(tex, n_fft=sz, hop_length=hp, window=wn)

In [50]:
IBM = 1*(trS > trN)

In [74]:
D = cdist(abs(teX).T,trX.T,'cosine')

In [83]:
Ds, DsIdx = np.sort(D,1), np.argsort(D,1)

In [84]:
K = 5

In [88]:
kNNIdx = DsIdx[:, :K]
kNNIdx1D = np.reshape(kNNIdx, -1)

In [103]:
IBM3D = IBM[:,kNNIdx1D]
IBM3D = np.reshape(IBM3D, [513, K, -1])
IBMEstMed = np.median(IBM3D, 1)
IBMEstMean = np.mean(IBM3D, 1)

In [114]:
yhat1 = librosa.istft(teX*IBMEstMed, hop_length=hp, window=wn)
yhat2 = librosa.istft(teX*IBMEstMean, hop_length=hp, window=wn)

In [116]:
def SDR(s,sr):
    eps=1e-20
    ml=np.minimum(len(s), len(sr))
    s=s[:ml]
    sr=sr[:ml]
    return ml, 10*np.log10(np.sum(s**2)/(np.sum((s-sr)**2)+eps)+eps)

In [117]:
SDR(yhat1, yhat2)

(66560, 5.351284471964779)

In [120]:
SDR(tes, tex)

(66560, -0.9322031636986516)